In [ ]:
# Imports here
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict
from workspace_utils import active_session

# Loading and transforming the data

In [ ]:
# transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])

valid_transforms = test_transforms
# Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform = train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform = valid_transforms)
test_data = datasets.ImageFolder(test_dir, transform = test_transforms)

# defining the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size = 32, shuffle = True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size = 32)
testloader = torch.utils.data.DataLoader(test_data, batch_size = 32)


# Building and training the model

In [ ]:
# load pretrained model
model = models.densenet121(pretrained = True)
# freeze all parameters
for param in model.parameters():
    param.requires_grad = False

In [ ]:
# write a new classifier and detatch the classifier of the pretrained network
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier = nn.Sequential(nn.Dropout(p=0.2),
                           nn.Linear(1024, 400),
                           nn.ReLU(),
                           nn.Dropout(p=0.2),
                           nn.Linear(400, 200),
                           nn.ReLU(),
                           nn.Dropout(p=0.2),
                           nn.Linear(200, 102),
                           nn.LogSoftmax(dim = 1))
model.classifier = classifier
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr = 0.003)
model.to(device)

In [ ]:
# Train the neural network
epochs = 1
steps = 0
print_every = 5
running_loss = 0

for epoch in range(epochs):
    for images, labels in trainloader:
        steps +=1
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        logps = model(images)
        loss = criterion(logps, labels)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()

            with torch.no_grad():
                for images, labels in validloader:
                    images, labels = images.to(device), labels.to(device)
                    logps = model(images)

                    valid_loss += criterion(logps, labels).item()

                    #accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim = 1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuarcy += torch.mean(equals.type(torch.FloatTensor))

            print("Validation loss: {}".format(valid_loss/len(validloader)))
            print("Validation Accuracy: {}".format(accuracy/len(validloader)))
            model.train()

# Testing

In [ ]:
#testing
model.eval()
                
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        logps = model(images)

        test_loss += criterion(logps, labels).item()

        #accuracy
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim = 1)
        equals = top_class == labels.view(*top_class.shape)
        accuarcy += torch.mean(equals.type(torch.FloatTensor))

print("Validation loss: {}".format(test_loss/len(validloader))
     "Validation Accuracy: {}".format(accuracy/len(validloader)))
model.train()

# Saving and loading the model

In [ ]:
class_to_name = cat_to_name

#train_data.class_to_idx
model.class_to_idx = train_data.class_to_idx
model.class_to_name = class_to_name


# TODO: Save the checkpoint 
checkpoint = {"transfer_model": "vgg13",
             "classifier": model.classifier,
             "classifier_state_dict": model.classifier.state_dict(),
             "optimizer": optimizer,
             "optimizer_state_dict": optimizer.state_dict(),
             "class_to_idx": model.class_to_idx,
             "class_to_name": model.class_to_name}
torch.save(checkpoint, "checkpoint_vgg13.pth")

In [ ]:
# loading function
def load_checkpoint(filepath):
    '''
    input: filepath to a checkpoint file -.pth
    output: 
        model - pretrained network with frozen parameters and a trained classifier with loaded state dict
        criterion - the criterion to be used
        optimizer - the optimizer with loaded state dict
    
    returns a trained model ready for predictions with a criterion and optimizer in case more training is needed
    '''
    device = ("cuda" if torch.cuda.is_available() else "cpu")
    
    checkpoint = torch.load(filepath)
    
    model = models.vgg13(pretrained = True)
    model.to(device)
    for param in model.parameters():
        param.requires_grad = False
        
    model.classifier = checkpoint["classifier"]
    model.classifier.load_state_dict(checkpoint["classifier_state_dict"])
    
    optimizer = checkpoint["optimizer"]
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    model.class_to_name = checkpoint["class_to_name"]
    model.class_to_idx = checkpoint["class_to_idx"]
    criterion = nn.NLLLoss()
   
    
    return model, criterion, optimizer

In [ ]:
model, criterion, optimizer = load_checkpoint("checkpoint_vgg13.pth")

# Image handling for prediction

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    transform = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])
   
    return transform(image)

In [ ]:
import matplotlib.pyplot as plt
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

# Predicting

In [ ]:
def predict(image_path, model = model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    model_check = "checkpoint_vgg13.pth"
   
    # loads and preprocesses the image
    image = Image.open(image_path, "r")
    image = process_image(image)
    
    # get the input tensor in the correct dimension for the model
    image = image.view(1,3,224,224)
    
    # don't load the model again if it is already loaded to save time
    try:
        
        model.to("cpu")
        model.eval()
        ps = torch.exp(model(image))
    except:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        model, criterion, optimizer = load_checkpoint(model)
        model.eval()
        ps = torch.exp(model(image))
        
        
    #get the top 5 predictions
    top_p, top_class = ps.topk(5, dim = 1)
    # convert tensor to a list
    top_p = top_p.detach().numpy()
    top_p=list(top_p.flatten())
    classes = list(top_class.detach().numpy().flatten())
    
    # map the predictions to the classes
    idx_to_class = {v: k for k, v in model.class_to_idx.items()}
    classes = list(top_class.detach().numpy().flatten())
    class_preds = []
    for item in classes:
        item = idx_to_class[item]
        class_preds.append(model.class_to_name[item])
        
    return top_p, class_preds

# Sanity check

In [ ]:
img_path = "flowers/test/28/image_05214.jpg"
image = Image.open(img_path)
label = [int(s) for s in img_path.split("/") if s.isdigit()]

label = model.class_to_name[str(label[0])]
idx_to_class = {v: k for k, v in model.class_to_idx.items()}


imshow(process_image(image))
preds, classes = predict(img_path, model)

fig, ax = plt.subplots()

y_pos = np.arange(len(classes))
performance = 3 + 10 * np.random.rand(len(classes))

ax.barh(y_pos, preds, align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(classes)
ax.invert_yaxis() 
ax.set_xlabel('Performance')
ax.set_title('Correct label: {} Predictions:'.format(label))

plt.show()